In [ ]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import base64
import json
from IPython.display import IFrame
from IPython.core.display import display

In [ ]:
display(IFrame('frame.html', '600px', '400px'))

In [ ]:
def save_image(payload):
    """Save latest image as jpg file for debugging only."""
    events = payload.collect()
    if len(events) > 0:
        latest = events[-1]
        decoded = base64.b64decode(latest['image'])  # Decode image from json
        filename = latest['camera_id'] + '.jpg' 
        with open(filename, 'wb') as f:
            f.write(decoded)

In [ ]:
# Stream Settings
zkQuorum = 'localhost:2181'
topic = 'pycturestream'
ssc = StreamingContext(sc, 3)  # Update every 3 seconds
kvs = KafkaUtils.createStream(ssc, 
                              zkQuorum, 
                              "spark-streaming-consumer", 
                              {topic: 1})

# Transform stream payload
dstream = kvs.map(lambda x: json.loads(x[1]))  # transform json to object

dstream.foreachRDD(lambda x: save_image(x))

# Actually start Streaming Context
ssc.start()  
ssc.awaitTermination()